# The Little Book of Semaphores

Working through my own implementations of concurrency and synchronization problems from the [Little Book of Semaphores](http://greenteapress.com/semaphores/LittleBookOfSemaphores.pdf) by A. B. Downey.

**Asyncio solutions in notebook**

We will implement examples using asyncio. 

**Reference solutions using Sync GUI app**

Solutions are also available from https://github.com/AllenDowney/LittleBookOfSemaphores/tree/master/code/sync_code and can be run using the Sync program provided in that repository:
```
git clone git@github.com:AllenDowney/LittleBookOfSemaphores.git
cd LittleBookOfSemaphores/code
python Sync.py sync_code/signal.py
```
![signalling problem using Downey's Sync app](sync_app.png)
```

```

In [27]:
import asyncio
import threading
from concurrent.futures import ThreadPoolExecutor
import logging 
from IPython.core.debugger import set_trace


In [33]:
logger = logging.getLogger()
logger.setLevel(logging.INFO)

In [4]:
async def run_test(test_coroutine, attempts):
    """Run a test multiple times to make sure we don't get lucky."""
    [await test_coroutine(attempt) for attempt in range(attempts)]

## Basic patterns

### Signaling
**Signaling problem**
2 threads/coroutines having to coordinate to do an action in a particular order.

In [5]:
async def test_signaling(attempt):
    """Check the signaling approach serialises the threads so that actionA 
    takes place before actionB. 
    
    Push a value from coroutines A then one from coroutine B into a shared queue, 
    and make sure they've been pushed in that order. 
    """
    # Not using a Lock here because we want to signal/release before waiting/acquiring 
    # so we need a semaphore. 
    first_action_done = asyncio.Semaphore(0)
    queue = asyncio.Queue()
    
    async def push(value):
        await queue.put(value)

    async def coroutineA():
        """Do first action then signal to B that we're done"""
        asyncio.Task.current_task().name = "coroutineA"
        await push('A')
        first_action_done.release()

    async def coroutineB():
        asyncio.Task.current_task().name = "coroutineB"
        async with first_action_done:
            res = await push('B')


    await asyncio.gather(coroutineA(), coroutineB())
    res = [await queue.get() for _ in range(queue.qsize())]
    assert res == ['A', 'B'], f'Test failed for attempt {attempt}: got {res}'

In [6]:
await run_test(test_signaling, 100)

### Rendez Vous
Two threads must await each other before doing some action. 

In [7]:
async def test_rendez_vous(attempt):
    """Make sure action doesn't happen before a rendez vous. 
    
    In this test, the action is reading the key value pairs from a shared dictionary. 
    Each of the two coroutines adds a key-value pair to the dictionary before the rendez vous. 
    So if both coroutines have waited for the other one successfully, they should both 
    return the two same (key,value) pairs. 
    """
    b_has_arrived = asyncio.Semaphore(0)
    a_has_arrived = asyncio.Semaphore(0)
    shared_dict = {}
    lock = asyncio.Lock()  # a lock to protect updating the dictionary
    
    async def read_items_from_dict():
        """Return tuple of (key, value) pairs giving the items in the dictionary."""
        async with lock:
            res = tuple(sorted(shared_dict.items()))
        return res 

    async def coroutineA(key, value):
        """Store key value pair in shared dictionary, wait for rendez vous with B, 
        then read all key value pairs from dictionary. 
        """
        a_has_arrived.release()
        # put in a value in the queue
        async with lock:
              shared_dict[key] = value
        await b_has_arrived.acquire()
        res = await read_items_from_dict()
        return res

    async def coroutineB(key, value):
        """Store key value pair in shared dictionary, wait for rendez vous with A, 
        then read all key value pairs from dictionary. 
        """
        b_has_arrived.release()
        async with lock:
              shared_dict[key] = value
        await a_has_arrived.acquire()
        res = await read_items_from_dict()
        return res

    itemsA, itemsB = await asyncio.gather(
        coroutineA('A', 1), 
        coroutineB('B', 2)
    )
    expected_items = ('A', 1), ('B', 2)
    results = {k:v for k,v in locals().items() if k in ['attempt', 'itemsA', 'itemsB', 'expected_items']}
    assert itemsA == itemsB == expected_items, results

In [8]:
await run_test(test_rendez_vous, 10)

### Mutex
Protects access to critical regions. Mutal exclusion: two threads can't both have access to the mutex at the same time. 

How to test:
- Use a lot of threads and make them race hard: see `test_mutex_threadpoool`
- Better demonstrated without asyncio as the event loop tends to make some of these issues go away because operations between awaits are atomic (there's only one thread). 
  - we can demonstrate with asyncio by forcing a context switch: see `test_mutex_aio`
  - we also can also run the threadpool test using aio `test_mutex_threadpool_aio`
 

#### Thread pool test

In [9]:
import random
import time
from concurrent.futures import ThreadPoolExecutor
import threading

def test_mutex_threadpool():
    """Check we can protect a variable using a mutex. 
    
    Better demonstrated with threads rather than coroutines because with asyncio's event loop
    operations between "awaits" are atomic.
    
    Use a big threadpool and make the threads race hard 
    by all incrementing the counter many times.
     
    """
    max_workers=100
    pool = ThreadPoolExecutor(max_workers=max_workers)
    counter = 0
    lock = threading.Lock()
    
    iterations = 100_000
    def increment_shared_counter(_):
        nonlocal counter
        with lock:
            for _ in range(iterations):
                counter += 1
        #time.sleep(random.random())
        return counter
    intermediate_results = list(pool.map(increment_shared_counter, range(max_workers)))
    #print(sorted(intermediate_results))
    assert counter == iterations * max_workers, {
        k:v for k,v in locals().items() if k in 
        ['counter', 'intermediate_results']
    }
    
test_mutex_threadpool() 

#### Thread pool + asyncio test

In [10]:
async def test_mutex_threadpool_aio():
    """Check we can protect a variable using a mutex. 
    
    Use a big threadpool and make the threads race hard 
    by all incrementing the counter many times. 
    
    Use asyncio to run the experiment via loop.run_in_executor 
    """
    pool = ThreadPoolExecutor(max_workers=100)
    lock = threading.Lock()
    counter = 0

    def increment_shared_counter(_):
        nonlocal counter
        with lock:
            for _ in range(100_000):
                counter = counter + 1
        #time.sleep(random.random())
        return counter
    
    loop = asyncio.get_event_loop()
    tasks = [
        loop.run_in_executor(pool, increment_shared_counter, i) 
        for i in range(100)
    ]
    intermediate_results = await asyncio.gather(*tasks)
    #print(sorted(intermediate_results))
    assert counter == 100_000 * 100, {
        k:v for k,v in locals().items() if k in 
        ['counter', 'intermediate_results']
    }

In [11]:
await test_mutex_threadpool_aio()

#### Minimal asyncio test

In [12]:
async def test_mutex_aio():
    """Check we can protect a variable using a mutex. 
    
    Minimal test with asyncio to force a context switch between 
    reading and updating the counter. 
    """
    counter = 0
    lock = asyncio.Lock()
    
    async def increment_shared_counter():
        nonlocal counter
        async with lock:   # fails 
            # this is a bit convoluted but is a simple way to 
            # simulating non-thread safe concurrent updates to counter:
            # we're sleeping between reading and incrementing the value of counter
            # which allows a context switch: without the lock this will fail the test
            x = counter
            # forcing a context switch by sleeping 
            await asyncio.sleep(0.001)
            counter = x + 1
        return counter
    tasks = [increment_shared_counter() for _ in range(2)]
    intermediate_results = await asyncio.gather(*tasks)
    #print(sorted(intermediate_results))
    assert counter == 1 * 2, {
        k:v for k,v in locals().items() if k in 
        ['counter', 'intermediate_results']
    }

In [13]:
await test_mutex_aio()

### Multiplex
A kind of bounded mutex where up to a certain number of threads are allowed to own a lock. 

Think of a bouncer who allows up to certain number of people to get into a nightclub (or a supermarket during Covid 19 outbreak), and bounces people off when the max capacity has been reached. Then whenever someone leaves the room, another person is allowed to get in.

In [14]:
async def test_multiplex(attempt):
    """Check the max number of active threads in the critical region is as expected.
    
    Use a counter and a variable to keep track of the max number of threads, and 
    verify it's <= the size of the multiplex. 
    """
    multiplex = asyncio.Semaphore(5)
    # we're using asyncio so no real need for locks 
    # to protect these variables: operations on them will be atomic 
    # in between awaits. 
    counter = 0
    max_counter = 0
    
    async def multiplexed(name):
        nonlocal counter
        nonlocal max_counter
        async with multiplex:
            counter += 1    # enterning critical zone 
            if counter >= max_counter:
                max_counter = counter
            await asyncio.sleep(0.1)  # block to allow context switch
            counter -= 1  # exiting critical zone 
        return max_counter
    tasks = [multiplexed(name) for name in range(7)]
    intermediate_results = await asyncio.gather(*tasks)
    assert max_counter == 5, {
        k:v for k,v in locals().items() if k in 
        ['max_counter', 'intermediate_results', 'attempt']
    }

In [15]:
await run_test(test_multiplex, 10)

In [16]:
loop = asyncio.get_event_loop()

### Barrier
Generalised rendez vous: all coroutines have to wait at the barrier until all of them have arrived. 



In [17]:
async def test_barrier(attempt):
    """All coroutines must wait at the barrier until they're all waiting behind it. 
    Then the barrier opens. 
    
    Testing strategy: all coroutines add their own key,value pair to a shared dictionary. 
    Once a coroutine passes the barrier, it retrieves all key value pairs from the dictionary. 
    If the barrier's been effective, all coroutines should read the same content made of one key value pair per coroutine. 
    """
    barrier = asyncio.Semaphore(0)  
    barrier_size = 5         # open barrier if nb threads behind barrier reaches this count
    lock = asyncio.Lock()    # to protect access to dictionary
    shared_dict = {}
    arrived_count = 0
    
    async def read_items_from_dict():
        """Return tuple of (key, value) pairs giving the items in the dictionary."""
        async with lock:
            res = tuple(sorted(shared_dict.items()))
        return res
    
    async def push_wait_then_read(key, value):
        nonlocal arrived_count
        async with lock:
            shared_dict[key] = value 
        arrived_count += 1
        if arrived_count == barrier_size:
            barrier.release()
        else:
            await barrier.acquire()
            barrier.release()
        items = await read_items_from_dict()
        return items 
    
    expected_items = tuple((key, value) for key, value in zip(list('ABCDE'), range(5)))
    tasks = [push_wait_then_read(key, value) for (key, value) in expected_items]
    received_items_per_coroutine = await asyncio.gather(*tasks)
    assert all(
        (received_items == expected_items) 
        for received_items 
        in received_items_per_coroutine
    ), {
        k:v for k,v in locals().items() if k in 
        ['received_items_per_coroutine', 'attempt']
    }
        

In [18]:
await run_test(test_barrier, 10)

### Reusable Barrier
Generalised rendez vous: all coroutines have to wait at the barrier until all of them have arrived. Barrier is in a loop so needs to be reused. 



In [19]:
async def test_reusable_barrier(attempt):
    """All coroutines must wait at the barrier until they're all waiting behind it. 
    Then the barrier opens. 
    
    Testing strategy: all coroutines add their own key,value pair to a shared dictionary. 
    Once a coroutine passes the barrier, it retrieves all key value pairs from the dictionary. 
    If the barrier's been effective, all coroutines should read the same content made of one key value pair per coroutine. 
    """
    turnstile = asyncio.Semaphore(0)  
    barrier_size = 5         # open barrier if nb threads behind barrier reaches this count
    lock = asyncio.Lock()    # to protect access to dictionary
    shared_dict = {}
    arrived_count = 0
    released_count = 0
    
    async def read_items_from_dict():
        """Return tuple of (key, value) pairs giving the items in the dictionary."""
        async with lock:
            res = tuple(sorted(shared_dict.items()))
        return res
    
    async def push_wait_then_read(key, value):
        nonlocal arrived_count
        nonlocal released_count 
        async with lock:
            shared_dict[key] = value 
        arrived_count += 1
        if arrived_count == barrier_size:
            turnstile.release()
            released_count += 1
        else:
            await turnstile.acquire()
            released_count += 1 
            if released_count < barrier_size:
                turnstile.release()
            else:
                # we're done: last thread exiting the turnstile,
                # so let's reset the counters so we can reuse 
                # the barrier again 
                arrived_count = 0
                released_count = 0
        items = await read_items_from_dict()
        return items 
    
    expected_items = tuple((key, value) for key, value in zip(list('ABCDE'), range(5)))
    # first time 
    tasks1 = [push_wait_then_read(key, value) for (key, value) in expected_items]
    received_items_per_coroutine1 = await asyncio.gather(*tasks1)
    assert all(
        (received_items == expected_items) 
        for received_items 
        in received_items_per_coroutine1
    ), {
        k:v for k,v in locals().items() if k in 
        ['received_items_per_coroutine1', 'attempt']
    }
    # second time 
    shared_dict = {}  # clear the dict
    tasks2 = [push_wait_then_read(key, value) for (key, value) in expected_items]
    received_items_per_coroutine2 = await asyncio.gather(*tasks2)
    
    assert all(
        (received_items == expected_items) 
        for received_items 
        in received_items_per_coroutine2
    ), {
        k:v for k,v in locals().items() if k in 
        ['received_items_per_coroutine2', 'attempt']
    }
        

In [20]:
await run_test(test_reusable_barrier, 10)

### Queues

Semaphores can be used to implement queues: use a mutex to protect access to a list, then 
append to it. To consume the queue, pop it. 

In [46]:
async def test_queue(attempt):
    """Rolling our own implementation of a queue using a list and a mutex. 
    
    Test: even though the coroutines will be launched together, we'll 
    use a counter to ensure the values are enqueued in a specific order. 
    Then we'll read from the queue and check we're getting the items 
    back in the expected order. 
    
    """
    lock = asyncio.Lock()
    counter = 0
    queue = []  # FIFO
    

    async def append_items_to_queue(index, value):
        """Store key value pair in shared dictionary, wait for rendez vous with B, 
        then read all key value pairs from dictionary. 
        """
        nonlocal counter
        while True:
            if counter == index:
                async with lock:
                    logger.debug('coroutine %d appending %d to queue.', index, value)
                    queue.append(value)
                counter += 1
                break
            else:
                logger.debug('coroutine %d waiting for turn.', index)
                await asyncio.sleep(0.001)

    expected = tuple(i*i for i in range(10))
    await asyncio.gather(
        *[append_items_to_queue(index, value) for index, value in enumerate(expected)]
    )
    logging.debug('Reading items from queue, %s.', queue)
    result = tuple(queue)
    assert result == expected, {
        k:v for k,v in locals().items() if k in 
        ['result', 'expected', 'attempt']
    }

In [49]:
#logger.setLevel(logging.DEBUG)
#await test_queue(0)

In [56]:
await run_test(test_queue, 10)

## Classical problems

### Producers and Consumers

- Producers push parameters to queue ready for consumption
- Consumers read paramters from queue 

### Readers and Writers
- if readers are in critical region, writers can't write 
- only one writer at a time in critical region
- multiple readers at a time allowed in critical retion

- mutex: protects access to critical region.
- counter: keeps track of number of readers in critical region

In [138]:
async def test_readers_and_writers(attempt):
    """
    
    Test 1:
    - launch a writer
    - launch a bunch of readers
    - verify that readers wait until the writer has left the critical zone.
    
    Test 2:
    - launch a bunch of readers
    - launch a writer
    - verify that the writer waits until the readers have left the critical zone.
    
    Test 3:
    - verify that you don't get two writers in critical zone at the same time. 
    """
    
    critical_region_empty = asyncio.Lock()
    counter_mutex = asyncio.Lock()
    nb_readers_in_critical_region = 0
    critical = []
    
    async def do_writing(x, writer_id):
        await asyncio.sleep(0.5)
        logger.debug(f'writer {writer_id} appending {x} to {critical}')
        critical.append(x)
        return critical
        
    async def do_reading(reader_id):
        await asyncio.sleep(random.random() * 0.1)
        logger.debug(f'reader {reader_id} reading from {critical}')
        return list(critical)
        
    async def writer(writer_id, value, sleep=0):
        await asyncio.sleep(sleep)
        async with critical_region_empty:
            logger.debug(f'writer {writer_id} entering critical region.')
            result = await do_writing(value, writer_id)
            logger.debug(f'writer {writer_id} leaving critical region.')
            return result
    
    async def reader(reader_id, sleep=0):
        await asyncio.sleep(sleep)
        nonlocal nb_readers_in_critical_region
        logger.debug('reader %s waiting for counter mutex', reader_id)
        async with counter_mutex:
            if nb_readers_in_critical_region == 0:
                # first reader blocks if there's a writer in critical region
                await critical_region_empty.acquire()
                logger.debug('reader %s locking critical region', reader_id)

        logger.debug(f'reader {reader_id} entering critical region.')
        nb_readers_in_critical_region += 1
        logger.debug(f'{nb_readers_in_critical_region} reader(s) in critical region.')

        result = await do_reading(reader_id)
        # leave critical region
        nb_readers_in_critical_region -= 1
        logger.debug('reader %s leaving critical region', reader_id)
        if nb_readers_in_critical_region == 0:
            # last reader out of critical region releases the lock
            logger.debug('reader %s releasing critical region lock', reader_id)
            critical_region_empty.release()
        return result 
        
    async def test_writer_first():
        nonlocal critical
        critical = []
        short_wait = 1e-5
        computed = await asyncio.gather(*[
            writer(0, 18),
            reader(0, short_wait), 
            reader(1, short_wait), 
            reader(2, short_wait)
        ])
        # if readers waited they should all have gotten 18. 
        expected = [[18]] * 4  

        assert computed == expected, {
            k:v for k,v in locals().items() if k in 
            ['computed', 'expected', 'attempt']
        }
        assert computed == expected 

    async def test_readers_first():
        nonlocal critical
        critical = [42]
        short_wait = 1e-5
        computed = await asyncio.gather(*[
            reader(0), 
            reader(1), 
            reader(2),
            writer(0, 18, short_wait),

        ])
        # if readers waited they should all have gotten 18. 
        expected = [[42]] * 3 + [[42, 18]]  

        assert computed == expected, {
            k:v for k,v in locals().items() if k in 
            ['computed', 'expected', 'attempt']
        }
        assert computed == expected 
       
    await test_readers_first()
            
            

In [139]:
logging.getLogger().setLevel(logging.DEBUG)
await test_readers_and_writers(0)

DEBUG:root:reader 1 waiting for counter mutex
DEBUG:root:reader 1 locking critical region
DEBUG:root:reader 1 entering critical region.
DEBUG:root:1 reader(s) in critical region.
DEBUG:root:reader 2 waiting for counter mutex
DEBUG:root:reader 2 entering critical region.
DEBUG:root:2 reader(s) in critical region.
DEBUG:root:reader 0 waiting for counter mutex
DEBUG:root:reader 0 entering critical region.
DEBUG:root:3 reader(s) in critical region.
DEBUG:root:reader 2 reading from [42]
DEBUG:root:reader 2 leaving critical region
DEBUG:root:reader 0 reading from [42]
DEBUG:root:reader 0 leaving critical region
DEBUG:root:reader 1 reading from [42]
DEBUG:root:reader 1 leaving critical region
DEBUG:root:reader 1 releasing critical region lock
DEBUG:root:writer 0 entering critical region.
DEBUG:root:writer 0 appending 18 to [42]
DEBUG:root:writer 0 leaving critical region.


### Dining Philosophers
- 5 philosophers, all executing the following loop
```python
while  True:
    think ()
    get_forks()
    eat()
    put_forks ()
```
- philosophers need 2 forks to eat
- loop stops once all philosophers have eaten
- only one philosopher can own a particular fork at any given time
- philosphers can't starve
- no dealock

In [140]:
def left(i):
    return i

def right(i):
    return (i + 1) % 5

In [188]:
async def test_dining_philosophers(attempt):
    
    forks = [asyncio.Lock() for _ in range(5)]
    _fork_location=[None] * 5   # None means table 
    _has_eaten = [False] * 5
    mutex = asyncio.Lock()
    
    async def philosopher(i):
        while True:
            #logging.debug('has eaten = %s', _has_eaten)
            await think(i)
            await get_forks(i)
            await eat(i)
            await put_forks(i)
            if all(_has_eaten):
                break
            
    async def think(i):
        logging.debug('philosopher %s is thinking', i)
        await asyncio.sleep(random.random() * 0.1)
        
    
    async def eat(i):
        if not _has_eaten[i]:
            logging.debug('philosopher %s is eating', i)
            await asyncio.sleep(random.random() * 0.1)
            _has_eaten[i] = True
            logging.debug('has eaten = %s', _has_eaten)
        
    async def get_forks(i):
        # if has left, wait right
        # if has right, await left
        if _has_eaten[i]:
            return 
        logging.debug('philosopher %s waiting for forks', i)
        if random.random() < 0.5:
            await forks[left(i)].acquire()
            async with mutex:
                _fork_location[left(i)] = i
            await forks[right(i)].acquire()
            async with mutex:
                _fork_location[right(i)] = i
        else:
            await forks[right(i)].acquire()
            async with mutex:
                _fork_location[right(i)] = i
            await forks[left(i)].acquire()
            async with mutex:
                _fork_location[left(i)] = i
        logger.debug('fork location = %s', _fork_location)
    
    async def put_forks(i):
        if _has_eaten[i]:
            async with mutex:
                if _fork_location[left(i)] == i:
                    _fork_location[left(i)] = None
                    forks[left(i)].release()
            async with mutex:
                if _fork_location[right(i)] == i:
                    _fork_location[right(i)] = None
                    forks[right(i)].release()
                
    
    
    tasks = [philosopher(i) for i in range(5)]
    await asyncio.gather(*tasks)

/Users/sinayoks/apps/miniconda3/envs/vaex/lib/python3.6/site-packages/IPython/core/compilerop.py:101: RuntimeWarning: coroutine 'test_dining_philosophers' was never awaited
  return compile(source, filename, symbol, self.flags | PyCF_ONLY_AST, 1)


In [191]:
%pdb on
logging.getLogger().setLevel(logging.INFO)
await run_test(test_dining_philosophers, 5)

Automatic pdb calling has been turned ON
